# RUN FOR FILES STEP 1

In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from datetime import datetime

def getDate():
    today = datetime.today()
    return str(today.strftime('%Y-%m-%d'))
    
today = getDate()

# Creates a url dataframe
df_urls = pd.read_excel("./url_data.xlsx")

df_urls.head()

## Metrics per Social Media Platform

In [ ]:
# Facebook
j = 0
facebook_urls = []

for i in df_urls['url_link']:

    if df_urls['platform'][j] == "Facebook":

        facebook_urls.append(i)

    j = j+1
    
# Twitter
j = 0
twitter_urls = []

for i in df_urls['url_link']:

    if df_urls['platform'][j] == "Twitter":

        twitter_urls.append(i)
    j = j+1

# LinkedIn
j = 0
linkedin_urls = []

for i in df_urls['url_link']:

    if df_urls['platform'][j] == "LinkedIn":

        linkedin_urls.append(i)

    j = j+1

# Youtube 
j = 0
youtube_urls = []

for i in df_urls['url_link']:

    if df_urls['platform'][j] == "Youtube":
        print(df_urls["platform"][j])
        youtube_urls.append(i)

    j = j+1

# Instagram
j = 0
instagram_urls = []

for i in df_urls['url_link']:

    if df_urls['platform'][j] == "Instagram":
        instagram_urls.append(i)

    j = j+1

## Webscraping
<hr>

### YouTube

#### Youtube functions
- scrapes youtube data
- returns an excel sheet

In [44]:
url = 'https://www.youtube.com/channel/UCnjKxBq90gADBOBAlDvdp6g'

def getYT(url=""):

    # Instantiates the webdriver, shwowing where the path to the chromedriver is
    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(url)

    # no cookies
    # driver.find_element(By.CLASS_NAME, "nCP5yc").click()
    driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/form[1]/div/div/button').click()
    time.sleep(2)


    # Makes the student hotel page pass over clicking on about due to different formatting
    if url != "https://www.youtube.com/user/TSHbroadcasting/about":

        # click on "ABOUT"
        driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[5]/div').click()
        time.sleep(2)

    # get number of followers
    try:
        # get number of followers
        subscribers = driver.find_element(By.XPATH, '//*[@id="subscriber-count"]').text

        if 'K' in subscribers:
            subscribers = re.sub("[^0-9]", "")
            subscribers = int(subscribers)
            subscribers =  1000  * subscribers
           
        else:
            subscribers = subscribers.split()[0]
            subscribers = int(subscribers)
    except:
        subscribers = np.nan

    # get total views (Aufrufe - German)
    views = driver.find_element(By.XPATH, '//*[@id="right-column"]/yt-formatted-string[3]').text
    views = views.split()[0]

    # 
    views = views.replace(",", "")
    views = int(views)
    

    # click on "VIDEOS" - might not work if more videos need to be loaded
    driver.find_element(By.XPATH, '//*[@id="tabsContent"]/tp-yt-paper-tab[2]/div').click()
    time.sleep(2)
        
    # get number of posts
    try:
        # get number of videos
        posts = driver.find_elements(By.XPATH, '''//*[@id="items"]//ytd-grid-video-renderer''')
        posts = len(posts)
        posts = int(posts)
    except:
        posts = np.nan

    # get ratio follower/posts
    try: 
        ratio_fp = subscribers/posts
    except:
        ratio_fp = np.nan

    driver.close()

    yt_metrices_dict = {
        "number_followers": subscribers, 
        "number_posts": posts, 
        "ratio_follower_posts": ratio_fp, 
        "total_video_views": views, 
    }
    
    return yt_metrices_dict

In [45]:
def youtube_gathering(youtube_urls):
        youtube_metrics= pd.DataFrame()

        for youtube_url in youtube_urls:  

                youtube_dictionary = getYT(youtube_url)
                youtube_metrics = youtube_metrics.append(youtube_dictionary, ignore_index=True)
                youtube_metrics.assign(date=today).set_index('date', append=True).T

        return youtube_metrics

In [ ]:
youtube_metrics = youtube_gathering(youtube_urls)
youtube_metrics.head()


### Twitter

#### Twitter functions

In [47]:
url = 'https://twitter.com/TheStudentHotel'

def getTwitter(url=""):

    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(url)

    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="layers"]/div/div[2]/div/div/div/div[2]/div[2]').click()

    try:
        num_followers = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[2]/div/div/div/div/div[5]/div[2]/a/span[1]/span').text
         # Replaces comma in any value
        if "," in num_followers:
            num_followers = num_followers.replace(",","")   
            num_followers = int(num_followers)

    except:
        num_followers = np.nan

    try:
        num_following = driver.find_element(By.XPATH, '//*[@id="react-root"]/div/div/div[2]/main/div/div/div/div/div/div[2]/div/div/div/div/div[5]/div[1]/a/span[1]/span').text
        
        if "." in num_following:
            num_following = num_following.replace(".", "")
            num_following = num_following[:-1]
            num_following = int(num_following)

        # num_following = float(num_following)
    except:
        num_following = np.nan

    try:
        # Attempt at finding number of tweets by boy
        num_tweets = driver.find_element(By.XPATH, '//*[@class="css-901oao css-1hf3ou5 r-1bwzh9t r-37j5jr r-n6v787 r-16dba41 r-1cwl3u0 r-bcqeeo r-qvutc0" ]').text

        if " Tweets" in num_tweets:
            num_tweets = re.sub("[^0-9]", "", num_tweets)
            num_tweets = int(num_tweets)
    except:
        num_tweets = np.nan

    

    driver.close()
    twitter_metrices_dict = {
        "number_followers": num_followers, 
        "number_following": num_following, 
        "number_tweets": num_tweets,         
    }


    
    
    return twitter_metrices_dict

In [48]:
# Takes url list 
# Requires the transform_instagram to have run first.
# Gathers data and returns pandas dataframe
def twitter_gathering(twitter_urls):
        twitter_metrics = pd.DataFrame()
        # Gathers, cleans and passes the data into a pandas dataframe
        for twitter_url in twitter_urls:

                # Maybe makes no sense to do
                twitter_row = {}

                # Returns a dictionary object
                twitter_dictionary = getTwitter(twitter_url)
                twitter_metrics = twitter_metrics.append(twitter_dictionary,ignore_index=True)
                twitter_metrics.assign(date=today).set_index("date", append=True).T
        
        # Temporary sleep, since instagram sometimes stops the scraping process
        time.sleep(3)

        # returns instagram excelsheet
        return twitter_metrics


### LinkedIn

#### LinkedIn functions

In [70]:
url = 'https://www.linkedin.com/company/the-student-hotel/'

def getLI(url=""):   
    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(url)

    time.sleep(1)


    # Make a function that either goes from sign up page or sign in page and then performs all functions



    # First click on sign in to change the webpage
    try:
        driver.find_element(By.XPATH, '//button[@class="authwall-join-form__form-toggle--bottom form-toggle"]').click()
        time.sleep(2)

        # Inputs username
        user_name = driver.find_element(By.XPATH, '//input[@autocomplete="username"]')
        user_name.send_keys("bt.stekelbos@hotmail.com")
  

        # Inputs Password
        time.sleep(1)
        password = driver.find_element(By.XPATH, '//input[@autocomplete="current-password"]')
  
        password.send_keys("TEMP-PASSWORD")
        time.sleep(1)
    

         # Clicks the sign in button
        driver.find_element(By.XPATH,'//button[@class="sign-in-form__submit-button"]').click()
        time.sleep(3)
    
        # Hop onto the specific page after logging in
        driver.get(url)


        # Scrape the relevant data which is only follower count
        num_followers = driver.find_element(By.XPATH,'//div[@class="inline-block"]').text
       
        # transforms the text info into number of followers
        number_followers = ''
        for i in num_followers: 
            if i.isdigit():
                number_followers = number_followers + i
        number_followers   = int(number_followers)

        num_followers = number_followers
    

        driver.close()
    
        li_metrices_dict = {
        "number_followers": [num_followers], 
        }

        return li_metrices_dict

        
    # If the landing page changes while looping through the url list it goes through this instead

    except:
       

        # If it lands on sign in page instead do the rest
        
        # <input id="username" name="session_key" type="text" aria-describedby="error-for-username"
        #  required="" validation="email|tel" class="" autofocus="" aria-label="Email or Phone">

        # Inputs username
        user_name = driver.find_element(By.XPATH, '//input[@oid="username"]')
        user_name.send_keys("bt.stekelbos@hotmail.com")
  
        # <input id="password" type="password" aria-describedby="error-for-password" name="session_password" required=""
        #  validation="password" autocomplete="current-password" class="" aria-label="Password">

        # Inputs Password
        time.sleep(1)
        password = driver.find_element(By.XPATH, '//input[@id="password"]')
  
        password.send_keys("weakpassword")
        time.sleep(1)
    

         # Clicks the sign in button
        driver.find_element(By.XPATH,'//button[@class="sign-in-form__submit-button"]').click()
        time.sleep(3)
    
        # Hop onto the specific page after logging in
        driver.get(url)


        # Scrape the relevant data which is only follower count
        num_followers = driver.find_element(By.XPATH,'//div[@class="inline-block"]').text
       
        # transforms the text info into number of followers
        number_followers = ''
        for i in num_followers: 
            if i.isdigit():
                number_followers = number_followers + i
        number_followers   = int(number_followers)

        num_followers = number_followers
    

        driver.close()
    
        li_metrices_dict = {
        "number_followers": num_followers, 
        }

        return li_metrices_dict

        # If it lands on sign in page instead do the rest
        

In [68]:
def linkedin_gathering(linkedin_urls):
        linkedin_metrics = pd.DataFrame()
        # Gathers, cleans and passes the data into a pandas dataframe
        for linkedin_url in linkedin_urls:
                
                # Returns a dictionary object
                linkedin_dictionary = getLI(linkedin_url)

                linkedin_metrics = linkedin_metrics.append(linkedin_dictionary,ignore_index=True)

                linkedin_metrics.assign(date=today).set_index("date", append=True).T

         # Temporary sleep
        time.sleep(3)

        # returns linkedin dataframe
        return linkedin_metrics

### Facebook

In [ ]:
# REWRITE THIS WHOLE THING

def facebook_transform(string_list):
     # 
    for value in string_list:
        
        # Checks for a k in the text and multiplies value by 1000 and adds to list
        if 'K' in value:
            numerical_instagram= re.sub("[^0-9]", "", value)
            numerical_instagram = int(numerical_instagram)
            value = numerical_instagram * 1000
            transformed_values_instagram.append(value)

        # transforms string into integer and adds to list object
        else:
            value = int(value)
            transformed_values_instagram.append(value)
        
    # Returns a list object
    return transformed_values_instagram

In [65]:
url = 'https://www.facebook.com/TheStudentHotelAmsterdamCity/?ref=page_internal'
information_list = []
def getFB(url=""):   
    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(url)

    # Create a function that clicks only essential cookies
    driver.find_element(By.XPATH, '//div[@aria-label="Only allow essential cookies"]').click()
    time.sleep(4)
    
    l = driver.find_elements(By.XPATH,"//*[@class='gvxzyvdx aeinzg81 t7p7dqev gh25dzvf tb6i94ri gupuyl1y i2onq4tn k1z55t6l oog5qr5w tes86rjd pbevjfx6']")
     # filters the list for index 3, 5 and 6, gathering all relevant information
    list_facebook = []
    for webelement in l:
        list_facebook.append(webelement.text)

    driver.close()

    list_2 = [list_facebook[0],list_facebook[2],list_facebook[3]]
        
    return list_2

In [ ]:
# not tested
for fb_url in links_fb[1:]:
    if (fb_url == None):
        pass
    else:
        new_fb = getFB(fb_url)
        
        for key, value in new_fb.items():
            test_fb[key].extend(value)

t_fb = pd.DataFrame(test_fb, index = [links_fb]) 
t_fb.assign(date=today).set_index("date", append=True).T


<span class="gvxzyvdx aeinzg81 t7p7dqev gh25dzvf tb6i94ri gupuyl1y i2onq4tn k1z55t6l oog5qr5w tes86rjd pbevjfx6">1,912 people</span>

### Instagram

#### Instagram functions
- Cleaning instagram data
- Scraping information
- Gathering all information and returning an excel file


In [71]:
# Takes scraped instagram data and cleans it
def transform_instagram(string_list):

    transformed_values_instagram = []


    # 
    for value in string_list:
        
        # Checks for a k in the text and multiplies value by 1000 and adds to list
        if 'K' in value:
            numerical_instagram= re.sub("[^0-9]", "", value)
            numerical_instagram = int(numerical_instagram)
            value = numerical_instagram * 1000
            transformed_values_instagram.append(value)

        # transforms string into integer and adds to list object
        else:
            value = int(value)
            transformed_values_instagram.append(value)
        
    # Returns a list object
    return transformed_values_instagram

In [72]:
url = 'https://www.instagram.com/thestudenthotel/'

# Creates a function that takes an URL as input,    
def getInsta(url=""):  
    
    #makes an object of the chrome driver function. 
    driver = webdriver.Chrome("./chromedriver")

    #passes the url into the object and opens the webpage
    driver.get(url)

    #forcibly passes time so the webpage doesnt force a time out
    time.sleep(3)
    
    # Create a function that clicks only essential cookies
    driver.find_element(By.XPATH, '//button[@class="_a9-- _a9_1"]').click()
    time.sleep(4)

    # Creates a list of web elements that have the class name _ac2a
    l = driver.find_elements(By.CLASS_NAME,'_ac2a')

    # Creates list and transform the webelements into text which get put into list_test
    list_test = []
    for i in l:
        list_test.append(i.text)

    # Transforms the string information into a list object with all relevant information
    clean_data = transform_instagram(list_test)

    post_effectiveness = clean_data[0] / clean_data[2]

    instagram_dict = {
    "posts": [clean_data[0]],
    "followers" : [clean_data[1]],
    "following" :[clean_data[2]],
    "post_effectivenss" :[post_effectiveness]
    }

    driver.close()

    return instagram_dict

In [74]:
# Takes url list 
# Requires the transform_instagram to have run first.
# Gathers data and returns pandas dataframe
def instagram_gathering(instagram_urls):

        instagram_metrics = pd.DataFrame()
    # Gathers, cleans and passes the data into a pandas dataframe
        for insta_url in instagram_urls:
                time.sleep(1)

                # Maybe makes no sense to do
                instagram_row = {}

                # Returns a dictionary object
                instagram_dictionary = getInsta(insta_url)

                # Creates a temporary dictionary looping over every item to get all info
                for key, value in instagram_dictionary.items():
                        instagram_row[key].extend(value)
        
        #------- WORK IN PROGRESS----------------#
                # RUN TOMORROW


                instagram_metrics = instagram_metrics.append(instagram_row, ignore_index = True)

                instagram_metrics.assign(date=today).set_index("date", append=True).T

        
        #------- WORK IN PROGRESS----------------#

    
        
        # Temporary sleep, since instagram sometimes stops the scraping process
        time.sleep(3)

        # returns dictionary
        return instagram_metrics



In [ ]:
# Loop over every url for instagram and add to pandas dataframe

# creates a dataframe to put all lists in

# loops over every url 
for insta_url in instagram_urls:

    # Returns a list object not a dict, so maybe return a dict instead and copy code that's already here
    new_insta = getInsta(insta_url)
    





    # for key, value in new_insta.items():
    #         test_insta[key].extend(value)
           

    transformed_values_instagram = []
    
    for value in new_insta:
        if 'K' in value:
            numerical_instagram= re.sub("[^0-9]", "", value)
            numerical_instagram = int(numerical_instagram)
            value = numerical_instagram * 1000
            transformed_values_instagram.append(value)

        else:
            

            value = value.replace(",","")
            value = int(value)
            transformed_values_instagram.append(value)
        

    
    instagram_dict = {}

    instagram_dict['posts'] = [transformed_values_instagram[0]]
    instagram_dict["followers"] = [transformed_values_instagram[1]]
    instagram_dict['following'] = [transformed_values_instagram[2]]

    print(instagram_dict)


    # Iterates but overrides the values in the pandas dataframe , so to fix make append true?

    instagram_df = pd.DataFrame.from_dict(instagram_dict,orient="columns")
    


    time.sleep(2)

instagram_df.head()
    


In [200]:
# Converts a full line of text into a dictionary that can be transformed or input into a dataframe
import re











instagram_split = test_insta.split(",")

numeric_instagram_split  = []
for list_value in instagram_split:
 


    numerical_instagram= re.sub("[^0-9]", "", list_value)
    numeric_instagram_split.append(numerical_instagram)
    
    # numeric_instagram_split.append(numerical_instagram)


print(numeric_instagram_split)

dictionary_keys = ["Number of followers","Number of following", "Number of posts"]

instagram_dictionary = dict(zip(dictionary_keys,numeric_instagram_split))




30K followers, 408 following, 901 posts – see Instagram photos and videos from The Student Hotel (@thestudenthotel)
['30', '408', '901']


In [ ]:
# not tested
# loops over the instagram url list, skips over the first urL? 


# REDO : RECREATE LINKS LIST INTO A CONGLORERATION OF ALL PLATFORM LINKS INTO ONE PANDAS DATAFRAME
# Loop over 
####for insta_url in df['instagram'][1:]

for insta_url in links_insta[1:]:
    # Why check whether there's no value when you're passing the value in there?
    if (insta_url == None):
        pass
    # Runs the function to gather information and returns a dictionairy, which new_insta inherits
    else:
        new_insta = getInsta(insta_url)
        
        # Loops over every metric of key and value within the dicitionairy 
        for key, value in new_insta.items():

            # Not sure what this does
            test_insta[key].extend(value)


# Makes a dataframe for the data gathered and indexes them based on url
t_insta = pd.DataFrame(test_insta, index = [links_insta]) 

#assigns time of date to the pandas dataframe
t_insta.assign(date=today).set_index("date", append=True).T

# Example of writing it out:
t_insta.to_excel("Instagram information.xlsx")




# TO DO:
# CHECK IF DIVS ARE WORKING 
# Facebook webscraping converts values into text but instagram scraping converts them into floats? why make that difference
# Get youtube webscraping to work and generate a pandas dataframe it breaks after clicking on the about button


# FIXED ----- Wrote a line to write the existing dataframe into an excel file
# Cant find any code to write this out into an excel file


# FIXED ---- Rewrote the database in excel and then transformed it into a pandas df, simple filter generates a url list now.
# Check the ranges with which the url links are being generated


# FIXED ---- Rewrote the database in excel and then transformed it into a pandas df, simple filter generates a url list now.
# Fix youtube url gathering from excel, its pulling linkedin url and only 4

# RUN FOR FILES STEP 2

NOTES:
- SHOULD EVENTUALLY RETURN AN EXCEL SHEET FILE WHICH AUTOMATTICALY STARTS DOWNLOADING
 

## Run for Instagram

In [ ]:
url = url = 'https://www.instagram.com/thestudenthotel/'
instagram = getInsta(url)

print(instagram)

In [78]:
driver = webdriver.Chrome("./chromedriver")
driver.get('https://www.instagram.com/accounts/login/?next=%2Fthestudenthotel%2F')



/var/folders/_1/99j0wl7d7l9btzc9rfh821t40000gn/T/ipykernel_5644/1098962209.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")


In [75]:
# SHOULD EVENTUALLY RETURN AN EXCEL SHEET FILE WHICH AUTOMATTICALY STARTS DOWNLOADING
instagram_metrics = instagram_gathering(instagram_urls)
instragam_metrics.head()
instagram_metrics.to_excel("Instagram_information.xlsx")

/var/folders/_1/99j0wl7d7l9btzc9rfh821t40000gn/T/ipykernel_5644/1890813414.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@class="_a9-- _a9_1"]"}
  (Session info: chrome=105.0.5195.125)
Stacktrace:
0   chromedriver                        0x0000000101191a90 chromedriver + 3889808
1   chromedriver                        0x0000000101120b54 chromedriver + 3427156
2   chromedriver                        0x0000000100e12238 chromedriver + 221752
3   chromedriver                        0x0000000100e434c8 chromedriver + 423112
4   chromedriver                        0x0000000100e6c944 chromedriver + 592196
5   chromedriver                        0x0000000100e3854c chromedriver + 378188
6   chromedriver                        0x00000001011652f8 chromedriver + 3707640
7   chromedriver                        0x0000000101168ea8 chromedriver + 3722920
8   chromedriver                        0x000000010116f3d8 chromedriver + 3748824
9   chromedriver                        0x00000001011699a8 chromedriver + 3

## Run for Youtube


In [ ]:
youtube_metrics = youtube_gathering(youtube_urls)
youtube_metrics.head()
youtube_metrics.to_excel("Youtube_information.xlsx")

## Run for Twitter

In [49]:
twitter_metrics = twitter_gathering(twitter_urls)
twitter_metrics.to_excel("Twitter_Information.xlsx")
twitter_metrics.head()

/var/folders/_1/99j0wl7d7l9btzc9rfh821t40000gn/T/ipykernel_5644/1078140683.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedriver")
/var/folders/_1/99j0wl7d7l9btzc9rfh821t40000gn/T/ipykernel_5644/755140697.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  twitter_metrics = twitter_metrics.append(twitter_dictionary,ignore_index=True)
/var/folders/_1/99j0wl7d7l9btzc9rfh821t40000gn/T/ipykernel_5644/1078140683.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedriver")
/var/folders/_1/99j0wl7d7l9btzc9rfh821t40000gn/T/ipykernel_5644/755140697.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  twitter_metrics = twitter

,number_followers,number_following,number_tweets
0,2683,688,NaN
1,194,140,NaN
2,489,494,NaN


## Run for LinkedIn

In [80]:
twitter_metrics.to_excel("Twitter_information.xlsm")

In [ ]:
linkedin_metrics = linkedin_gathering(linkedin_urls)
linkedin_metrics.head()
linkedin_metrics.to_excel("LinkedIn_information.xlsx")